# 📊 AI Fashion Assistant v2.0 - Monitoring & Analytics

**Phase 7, Notebook 3/3** - Production Monitoring & Observability

---

## 🎯 Objectives

1. **Prometheus Metrics:** Time-series monitoring
2. **Grafana Dashboards:** Visual monitoring
3. **Log Aggregation:** Centralized logging
4. **Alert Rules:** Automated alerting
5. **Performance Analytics:** System insights

---

## 📊 Monitoring Architecture

### **Metrics Stack:**
```
Application (FastAPI)
  ↓
Prometheus Exporter
  ↓
Prometheus (TSDB)
  ↓
Grafana (Visualization)
```

### **Logging Stack:**
```
Application Logs
  ↓
JSON Format
  ↓
File / Stdout
  ↓
Log Aggregation (ELK/Loki)
```

---

## 🎯 Quality Gates

- ✓ Prometheus metrics exposed
- ✓ Grafana dashboard created
- ✓ Alert rules defined
- ✓ Structured logging implemented
- ✓ Performance analytics ready
- ✓ SLA metrics tracked

---

In [1]:
# ============================================================
# 1) SETUP
# ============================================================

from google.colab import drive
drive.mount("/content/drive", force_remount=False)

print("✅ Drive mounted")

Mounted at /content/drive
✅ Drive mounted


In [2]:
# ============================================================
# 2) INSTALL MONITORING DEPENDENCIES
# ============================================================

print("📦 Installing monitoring dependencies...\n")

!pip install -q prometheus-client prometheus-fastapi-instrumentator

print("\n✅ Dependencies installed!")

📦 Installing monitoring dependencies...


✅ Dependencies installed!


In [3]:
# ============================================================
# 3) IMPORTS
# ============================================================

import os
import sys
from pathlib import Path
import json
import yaml
from datetime import datetime
from typing import Dict, List, Any

print("✅ Imports successful!")

✅ Imports successful!


In [4]:
# ============================================================
# 4) PROJECT PATHS
# ============================================================

PROJECT_ROOT = Path("/content/drive/MyDrive/ai_fashion_assistant_v2")
MONITORING_DIR = PROJECT_ROOT / "monitoring"
CONFIG_DIR = PROJECT_ROOT / "config"
DOCKER_DIR = PROJECT_ROOT / "docker"

# Create monitoring directory
MONITORING_DIR.mkdir(exist_ok=True)
(MONITORING_DIR / "prometheus").mkdir(exist_ok=True)
(MONITORING_DIR / "grafana").mkdir(exist_ok=True)
(MONITORING_DIR / "alerts").mkdir(exist_ok=True)

print("📁 Monitoring Structure:")
print(f"  Root: {MONITORING_DIR}")
print(f"  Prometheus: {MONITORING_DIR / 'prometheus'}")
print(f"  Grafana: {MONITORING_DIR / 'grafana'}")
print(f"  Alerts: {MONITORING_DIR / 'alerts'}")

📁 Monitoring Structure:
  Root: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring
  Prometheus: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/prometheus
  Grafana: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/grafana
  Alerts: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/alerts


In [5]:
# ============================================================
# 5) PROMETHEUS CONFIGURATION
# ============================================================

print("\n📊 CREATING PROMETHEUS CONFIG...\n")
print("=" * 80)

prometheus_config = '''# Prometheus Configuration
# AI Fashion Assistant v2.0

global:
  scrape_interval: 15s
  evaluation_interval: 15s
  external_labels:
    cluster: 'fashion-api'
    environment: 'production'

# Alert manager configuration
alerting:
  alertmanagers:
    - static_configs:
        - targets:
            - alertmanager:9093

# Load rules
rule_files:
  - '/etc/prometheus/alerts/*.yml'

# Scrape configurations
scrape_configs:
  # FastAPI Application
  - job_name: 'fashion-api'
    static_configs:
      - targets: ['api:8000']
    metrics_path: '/metrics'
    scrape_interval: 10s

  # Redis
  - job_name: 'redis'
    static_configs:
      - targets: ['redis:6379']

  # Nginx
  - job_name: 'nginx'
    static_configs:
      - targets: ['nginx:9113']

  # Node Exporter (system metrics)
  - job_name: 'node'
    static_configs:
      - targets: ['node-exporter:9100']
'''

prom_config_path = MONITORING_DIR / "prometheus" / "prometheus.yml"
with open(prom_config_path, 'w') as f:
    f.write(prometheus_config)

print(f"✅ Prometheus config: {prom_config_path}")
print("\n📋 Features:")
print("  - Scrape interval: 15s")
print("  - Evaluation interval: 15s")
print("  - Jobs: API, Redis, Nginx, Node")
print("  - Alert manager integration")

print("\n" + "=" * 80)
print("✅ Prometheus config ready!")


📊 CREATING PROMETHEUS CONFIG...

✅ Prometheus config: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/prometheus/prometheus.yml

📋 Features:
  - Scrape interval: 15s
  - Evaluation interval: 15s
  - Jobs: API, Redis, Nginx, Node
  - Alert manager integration

✅ Prometheus config ready!


In [6]:
# ============================================================
# 6) ALERT RULES
# ============================================================

print("\n🚨 CREATING ALERT RULES...\n")
print("=" * 80)

alert_rules = '''# Alert Rules
# AI Fashion Assistant v2.0

groups:
  - name: api_alerts
    interval: 30s
    rules:
      # High error rate
      - alert: HighErrorRate
        expr: |
          (
            rate(http_requests_total{status=~"5.."}[5m])
            /
            rate(http_requests_total[5m])
          ) > 0.05
        for: 5m
        labels:
          severity: critical
        annotations:
          summary: "High error rate detected"
          description: "Error rate is {{ $value | humanizePercentage }} (threshold: 5%)"

      # High latency
      - alert: HighLatency
        expr: |
          histogram_quantile(0.95,
            rate(http_request_duration_seconds_bucket[5m])
          ) > 1.0
        for: 5m
        labels:
          severity: warning
        annotations:
          summary: "High API latency detected"
          description: "P95 latency is {{ $value }}s (threshold: 1s)"

      # Service down
      - alert: ServiceDown
        expr: up{job="fashion-api"} == 0
        for: 1m
        labels:
          severity: critical
        annotations:
          summary: "Service is down"
          description: "Fashion API has been down for more than 1 minute"

      # High CPU usage
      - alert: HighCPUUsage
        expr: |
          100 - (avg by (instance) (
            irate(node_cpu_seconds_total{mode="idle"}[5m])
          ) * 100) > 80
        for: 10m
        labels:
          severity: warning
        annotations:
          summary: "High CPU usage"
          description: "CPU usage is {{ $value }}% (threshold: 80%)"

      # High memory usage
      - alert: HighMemoryUsage
        expr: |
          (
            node_memory_MemTotal_bytes - node_memory_MemAvailable_bytes
          ) / node_memory_MemTotal_bytes > 0.85
        for: 10m
        labels:
          severity: warning
        annotations:
          summary: "High memory usage"
          description: "Memory usage is {{ $value | humanizePercentage }} (threshold: 85%)"

      # Redis down
      - alert: RedisDown
        expr: redis_up == 0
        for: 1m
        labels:
          severity: critical
        annotations:
          summary: "Redis is down"
          description: "Redis cache has been down for more than 1 minute"
'''

alerts_path = MONITORING_DIR / "alerts" / "api_alerts.yml"
with open(alerts_path, 'w') as f:
    f.write(alert_rules)

print(f"✅ Alert rules: {alerts_path}")
print("\n📋 Alerts Configured:")
print("  - HighErrorRate (>5%)")
print("  - HighLatency (P95 >1s)")
print("  - ServiceDown (>1min)")
print("  - HighCPUUsage (>80%)")
print("  - HighMemoryUsage (>85%)")
print("  - RedisDown (>1min)")

print("\n" + "=" * 80)
print("✅ Alert rules ready!")


🚨 CREATING ALERT RULES...

✅ Alert rules: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/alerts/api_alerts.yml

📋 Alerts Configured:
  - HighErrorRate (>5%)
  - HighLatency (P95 >1s)
  - ServiceDown (>1min)
  - HighCPUUsage (>80%)
  - HighMemoryUsage (>85%)
  - RedisDown (>1min)

✅ Alert rules ready!


In [7]:
# ============================================================
# 7) GRAFANA DASHBOARD
# ============================================================

print("\n📊 CREATING GRAFANA DASHBOARD...\n")
print("=" * 80)

# Grafana dashboard JSON (simplified)
dashboard = {
    "dashboard": {
        "title": "AI Fashion Assistant - Production Monitoring",
        "tags": ["fashion", "api", "production"],
        "timezone": "browser",
        "panels": [
            {
                "id": 1,
                "title": "Request Rate",
                "type": "graph",
                "targets": [
                    {
                        "expr": "rate(http_requests_total[5m])",
                        "legendFormat": "{{method}} {{path}}"
                    }
                ]
            },
            {
                "id": 2,
                "title": "Error Rate",
                "type": "graph",
                "targets": [
                    {
                        "expr": "rate(http_requests_total{status=~'5..'}[5m])",
                        "legendFormat": "Errors"
                    }
                ]
            },
            {
                "id": 3,
                "title": "Latency (P50, P95, P99)",
                "type": "graph",
                "targets": [
                    {
                        "expr": "histogram_quantile(0.50, rate(http_request_duration_seconds_bucket[5m]))",
                        "legendFormat": "P50"
                    },
                    {
                        "expr": "histogram_quantile(0.95, rate(http_request_duration_seconds_bucket[5m]))",
                        "legendFormat": "P95"
                    },
                    {
                        "expr": "histogram_quantile(0.99, rate(http_request_duration_seconds_bucket[5m]))",
                        "legendFormat": "P99"
                    }
                ]
            },
            {
                "id": 4,
                "title": "CPU Usage",
                "type": "graph",
                "targets": [
                    {
                        "expr": "100 - (avg(irate(node_cpu_seconds_total{mode='idle'}[5m])) * 100)",
                        "legendFormat": "CPU %"
                    }
                ]
            },
            {
                "id": 5,
                "title": "Memory Usage",
                "type": "graph",
                "targets": [
                    {
                        "expr": "(node_memory_MemTotal_bytes - node_memory_MemAvailable_bytes) / node_memory_MemTotal_bytes * 100",
                        "legendFormat": "Memory %"
                    }
                ]
            },
            {
                "id": 6,
                "title": "Redis Operations",
                "type": "graph",
                "targets": [
                    {
                        "expr": "rate(redis_commands_total[5m])",
                        "legendFormat": "{{cmd}}"
                    }
                ]
            }
        ]
    }
}

dashboard_path = MONITORING_DIR / "grafana" / "dashboard.json"
with open(dashboard_path, 'w') as f:
    json.dump(dashboard, f, indent=2)

print(f"✅ Grafana dashboard: {dashboard_path}")
print("\n📋 Dashboard Panels:")
print("  1. Request Rate (QPS)")
print("  2. Error Rate (%)")
print("  3. Latency (P50, P95, P99)")
print("  4. CPU Usage (%)")
print("  5. Memory Usage (%)")
print("  6. Redis Operations")

print("\n" + "=" * 80)
print("✅ Grafana dashboard ready!")


📊 CREATING GRAFANA DASHBOARD...

✅ Grafana dashboard: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/grafana/dashboard.json

📋 Dashboard Panels:
  1. Request Rate (QPS)
  2. Error Rate (%)
  3. Latency (P50, P95, P99)
  4. CPU Usage (%)
  5. Memory Usage (%)
  6. Redis Operations

✅ Grafana dashboard ready!


In [8]:
# ============================================================
# 8) STRUCTURED LOGGING CONFIG
# ============================================================

print("\n📝 CREATING LOGGING CONFIG...\n")
print("=" * 80)

logging_config = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'json': {
            '()': 'pythonjsonlogger.jsonlogger.JsonFormatter',
            'format': '%(asctime)s %(name)s %(levelname)s %(message)s'
        },
        'standard': {
            'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        }
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'level': 'INFO',
            'formatter': 'json',
            'stream': 'ext://sys.stdout'
        },
        'file': {
            'class': 'logging.handlers.RotatingFileHandler',
            'level': 'INFO',
            'formatter': 'json',
            'filename': '/var/log/fashion_api.log',
            'maxBytes': 10485760,  # 10MB
            'backupCount': 5
        },
        'error_file': {
            'class': 'logging.handlers.RotatingFileHandler',
            'level': 'ERROR',
            'formatter': 'json',
            'filename': '/var/log/fashion_api_errors.log',
            'maxBytes': 10485760,
            'backupCount': 5
        }
    },
    'loggers': {
        'fashion_api': {
            'level': 'INFO',
            'handlers': ['console', 'file', 'error_file'],
            'propagate': False
        },
        'uvicorn': {
            'level': 'INFO',
            'handlers': ['console'],
            'propagate': False
        }
    },
    'root': {
        'level': 'INFO',
        'handlers': ['console']
    }
}

logging_path = CONFIG_DIR / "logging.yaml"
with open(logging_path, 'w') as f:
    yaml.dump(logging_config, f, default_flow_style=False)

print(f"✅ Logging config: {logging_path}")
print("\n📋 Features:")
print("  - Format: JSON (structured)")
print("  - Console: stdout (INFO)")
print("  - File: rotating (10MB, 5 backups)")
print("  - Error file: separate (ERROR+)")
print("  - Loggers: fashion_api, uvicorn")

print("\n" + "=" * 80)
print("✅ Logging config ready!")


📝 CREATING LOGGING CONFIG...

✅ Logging config: /content/drive/MyDrive/ai_fashion_assistant_v2/config/logging.yaml

📋 Features:
  - Format: JSON (structured)
  - Console: stdout (INFO)
  - File: rotating (10MB, 5 backups)
  - Error file: separate (ERROR+)
  - Loggers: fashion_api, uvicorn

✅ Logging config ready!


In [9]:
# ============================================================
# 9) DOCKER COMPOSE UPDATE (ADD MONITORING)
# ============================================================

print("\n🐳 UPDATING DOCKER COMPOSE (MONITORING)...\n")
print("=" * 80)

docker_compose_monitoring = '''version: '3.8'

services:
  # ... (existing services: api, redis, nginx)

  # Prometheus
  prometheus:
    image: prom/prometheus:latest
    container_name: fashion_prometheus
    restart: unless-stopped
    ports:
      - "9090:9090"
    volumes:
      - ../monitoring/prometheus:/etc/prometheus
      - prometheus_data:/prometheus
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'
      - '--storage.tsdb.path=/prometheus'
      - '--web.console.libraries=/usr/share/prometheus/console_libraries'
      - '--web.console.templates=/usr/share/prometheus/consoles'
    networks:
      - fashion_network

  # Grafana
  grafana:
    image: grafana/grafana:latest
    container_name: fashion_grafana
    restart: unless-stopped
    ports:
      - "3000:3000"
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
      - GF_USERS_ALLOW_SIGN_UP=false
    volumes:
      - grafana_data:/var/lib/grafana
      - ../monitoring/grafana:/etc/grafana/provisioning
    networks:
      - fashion_network
    depends_on:
      - prometheus

volumes:
  prometheus_data:
    driver: local
  grafana_data:
    driver: local
'''

compose_monitoring_path = DOCKER_DIR / "docker-compose.monitoring.yml"
with open(compose_monitoring_path, 'w') as f:
    f.write(docker_compose_monitoring)

print(f"✅ Docker Compose monitoring: {compose_monitoring_path}")
print("\n📋 Additional Services:")
print("  - Prometheus (Port 9090)")
print("  - Grafana (Port 3000)")
print("\n📊 Access:")
print("  Prometheus: http://localhost:9090")
print("  Grafana: http://localhost:3000 (admin/admin)")

print("\n" + "=" * 80)
print("✅ Docker Compose updated!")


🐳 UPDATING DOCKER COMPOSE (MONITORING)...

✅ Docker Compose monitoring: /content/drive/MyDrive/ai_fashion_assistant_v2/docker/docker-compose.monitoring.yml

📋 Additional Services:
  - Prometheus (Port 9090)
  - Grafana (Port 3000)

📊 Access:
  Prometheus: http://localhost:9090
  Grafana: http://localhost:3000 (admin/admin)

✅ Docker Compose updated!


In [10]:
# ============================================================
# 10) PERFORMANCE ANALYTICS SCRIPT
# ============================================================

print("\n📊 CREATING ANALYTICS SCRIPT...\n")
print("=" * 80)

analytics_script = '''#!/usr/bin/env python3
"""
Performance Analytics Script
AI Fashion Assistant v2.0
"""

import requests
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

PROMETHEUS_URL = "http://localhost:9090"

def query_prometheus(query, start, end, step='1m'):
    """Query Prometheus API."""
    params = {
        'query': query,
        'start': start.isoformat(),
        'end': end.isoformat(),
        'step': step
    }
    response = requests.get(f"{PROMETHEUS_URL}/api/v1/query_range", params=params)
    return response.json()

def get_request_rate(hours=24):
    """Get request rate for last N hours."""
    end = datetime.now()
    start = end - timedelta(hours=hours)
    query = 'rate(http_requests_total[5m])'
    return query_prometheus(query, start, end)

def get_error_rate(hours=24):
    """Get error rate for last N hours."""
    end = datetime.now()
    start = end - timedelta(hours=hours)
    query = 'rate(http_requests_total{status=~"5.."}[5m]) / rate(http_requests_total[5m])'
    return query_prometheus(query, start, end)

def get_latency_percentiles(hours=24):
    """Get latency percentiles."""
    end = datetime.now()
    start = end - timedelta(hours=hours)

    percentiles = {}
    for p in [50, 95, 99]:
        query = f'histogram_quantile({p/100}, rate(http_request_duration_seconds_bucket[5m]))'
        percentiles[f'p{p}'] = query_prometheus(query, start, end)

    return percentiles

def generate_report():
    """Generate performance report."""
    print("\\n📊 PERFORMANCE ANALYTICS REPORT")
    print("=" * 80)

    # Request rate
    print("\\n1. Request Rate (24h)")
    rate_data = get_request_rate()
    print(f"   Current: {rate_data} req/s")

    # Error rate
    print("\\n2. Error Rate (24h)")
    error_data = get_error_rate()
    print(f"   Current: {error_data}%")

    # Latency
    print("\\n3. Latency Percentiles (24h)")
    latency_data = get_latency_percentiles()
    for p, data in latency_data.items():
        print(f"   {p}: {data}ms")

    print("\\n" + "=" * 80)

if __name__ == '__main__':
    generate_report()
'''

analytics_path = MONITORING_DIR / "analytics.py"
with open(analytics_path, 'w') as f:
    f.write(analytics_script)
os.chmod(analytics_path, 0o755)

print(f"✅ Analytics script: {analytics_path}")
print("\n📋 Features:")
print("  - Query Prometheus API")
print("  - Request rate analysis")
print("  - Error rate tracking")
print("  - Latency percentiles (P50, P95, P99)")
print("  - 24h reports")

print("\n" + "=" * 80)
print("✅ Analytics script ready!")


📊 CREATING ANALYTICS SCRIPT...

✅ Analytics script: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/analytics.py

📋 Features:
  - Query Prometheus API
  - Request rate analysis
  - Error rate tracking
  - Latency percentiles (P50, P95, P99)
  - 24h reports

✅ Analytics script ready!


In [11]:
# ============================================================
# 11) MONITORING DOCUMENTATION
# ============================================================

print("\n📚 CREATING MONITORING DOCS...\n")
print("=" * 80)

monitoring_docs = '''# Monitoring & Observability Guide

## Overview

Complete monitoring stack for AI Fashion Assistant v2.0.

## Architecture

```
Application → Prometheus → Grafana
     ↓            ↓            ↓
  Metrics      Storage    Visualization
```

## Components

### Prometheus
- **Port:** 9090
- **Purpose:** Time-series metrics storage
- **Scrape Interval:** 15s
- **Retention:** 15 days

### Grafana
- **Port:** 3000
- **Purpose:** Visualization & dashboards
- **Default Login:** admin/admin
- **Dashboards:** Pre-configured

## Key Metrics

### Request Metrics
- `http_requests_total`: Total request count
- `http_request_duration_seconds`: Request latency
- `http_requests_in_progress`: Concurrent requests

### Error Metrics
- `http_requests_total{status="5xx"}`: Server errors
- `http_requests_total{status="4xx"}`: Client errors

### System Metrics
- `node_cpu_seconds_total`: CPU usage
- `node_memory_MemAvailable_bytes`: Available memory
- `node_disk_io_time_seconds_total`: Disk I/O

### Redis Metrics
- `redis_commands_total`: Redis command count
- `redis_connected_clients`: Client connections
- `redis_memory_used_bytes`: Memory usage

## Alert Rules

### Critical Alerts
- **ServiceDown:** API unavailable >1min
- **HighErrorRate:** Error rate >5% for 5min
- **RedisDown:** Cache unavailable >1min

### Warning Alerts
- **HighLatency:** P95 latency >1s for 5min
- **HighCPUUsage:** CPU usage >80% for 10min
- **HighMemoryUsage:** Memory >85% for 10min

## Usage

### Start Monitoring
```bash
cd docker
docker-compose -f docker-compose.yml -f docker-compose.monitoring.yml up -d
```

### Access Dashboards
- Prometheus: http://localhost:9090
- Grafana: http://localhost:3000

### Query Examples

**Request rate:**
```promql
rate(http_requests_total[5m])
```

**P95 latency:**
```promql
histogram_quantile(0.95, rate(http_request_duration_seconds_bucket[5m]))
```

**Error rate:**
```promql
rate(http_requests_total{status=~"5.."}[5m]) / rate(http_requests_total[5m])
```

## SLA Targets

### Availability
- **Target:** 99.9% (43.8 minutes downtime/month)
- **Measurement:** `up` metric

### Latency
- **P50:** <100ms
- **P95:** <500ms
- **P99:** <1000ms

### Error Rate
- **Target:** <1%
- **Critical:** >5%

## Troubleshooting

### Prometheus not scraping
1. Check `/metrics` endpoint accessibility
2. Verify network connectivity
3. Check Prometheus targets page

### Grafana dashboard empty
1. Verify Prometheus data source
2. Check time range selection
3. Verify metrics exist in Prometheus

### High latency
1. Check CPU/Memory usage
2. Review slow queries
3. Check Redis cache hit rate
4. Profile application code
'''

docs_path = MONITORING_DIR / "README.md"
with open(docs_path, 'w') as f:
    f.write(monitoring_docs)

print(f"✅ Monitoring docs: {docs_path}")
print("\n📋 Documentation Sections:")
print("  - Architecture overview")
print("  - Components (Prometheus, Grafana)")
print("  - Key metrics")
print("  - Alert rules")
print("  - Usage guide")
print("  - SLA targets")
print("  - Troubleshooting")

print("\n" + "=" * 80)
print("✅ Monitoring docs ready!")


📚 CREATING MONITORING DOCS...

✅ Monitoring docs: /content/drive/MyDrive/ai_fashion_assistant_v2/monitoring/README.md

📋 Documentation Sections:
  - Architecture overview
  - Components (Prometheus, Grafana)
  - Key metrics
  - Alert rules
  - Usage guide
  - SLA targets
  - Troubleshooting

✅ Monitoring docs ready!


In [13]:
# ============================================================
# 12) QUALITY GATES
# ============================================================

print("\n🎯 QUALITY GATES VALIDATION")
print("=" * 80)

gates_passed = 0
total_gates = 8

# Gate 1: Prometheus config
if (MONITORING_DIR / "prometheus" / "prometheus.yml").exists():
    print("✅ Gate 1: Prometheus config created")
    gates_passed += 1
else:
    print("❌ Gate 1: Prometheus config missing")

# Gate 2: Alert rules
if (MONITORING_DIR / "alerts" / "api_alerts.yml").exists():
    print("✅ Gate 2: Alert rules created (6 alerts)")
    gates_passed += 1
else:
    print("❌ Gate 2: Alert rules missing")

# Gate 3: Grafana dashboard
if (MONITORING_DIR / "grafana" / "dashboard.json").exists():
    print("✅ Gate 3: Grafana dashboard created (6 panels)")
    gates_passed += 1
else:
    print("❌ Gate 3: Grafana dashboard missing")

# Gate 4: Logging config
if (CONFIG_DIR / "logging.yaml").exists():
    print("✅ Gate 4: Logging config created (JSON format)")
    gates_passed += 1
else:
    print("❌ Gate 4: Logging config missing")

# Gate 5: Docker Compose monitoring
if (DOCKER_DIR / "docker-compose.monitoring.yml").exists():
    print("✅ Gate 5: Docker Compose monitoring created")
    gates_passed += 1
else:
    print("❌ Gate 5: Docker Compose monitoring missing")

# Gate 6: Analytics script
if (MONITORING_DIR / "analytics.py").exists():
    print("✅ Gate 6: Analytics script created")
    gates_passed += 1
else:
    print("❌ Gate 6: Analytics script missing")

# Gate 7: Documentation
if (MONITORING_DIR / "README.md").exists():
    print("✅ Gate 7: Monitoring docs created")
    gates_passed += 1
else:
    print("❌ Gate 7: Monitoring docs missing")

# Gate 8: Complete monitoring stack
dirs_exist = all([
    (MONITORING_DIR / "prometheus").exists(),
    (MONITORING_DIR / "grafana").exists(),
    (MONITORING_DIR / "alerts").exists()
])
if dirs_exist:
    print("✅ Gate 8: Monitoring directories structure complete")
    gates_passed += 1
else:
    print("❌ Gate 8: Some directories missing")

print("=" * 80)
print(f"\n📊 Gates Passed: {gates_passed}/{total_gates}")

if gates_passed >= 7:
    print("\n🎉 QUALITY GATES PASSED!")
    print("✅ Phase 7, Notebook 3 complete!")
else:
    print("\n⚠️ Some quality gates need attention")

print("\n📊 Summary:")
print(f"  Prometheus config: ✓")
print(f"  Alert rules: 6 ✓")
print(f"  Grafana dashboard: 6 panels ✓")
print(f"  Logging config: JSON ✓")
print(f"  Docker Compose: monitoring ✓")
print(f"  Analytics script: Python ✓")
print(f"  Documentation: Complete ✓")

print("\n📝 Monitoring Stack:")
print("""\n# Start monitoring:
cd docker
docker-compose -f docker-compose.yml -f docker-compose.monitoring.yml up -d

# Access:
Prometheus: http://localhost:9090
Grafana: http://localhost:3000 (admin/admin)

# Run analytics:
python monitoring/analytics.py

# View logs:
docker-compose logs -f api
""")

print("\n" + "=" * 80)
print("🎊 PHASE 7 COMPLETE! ALL 3 NOTEBOOKS DONE!")
print("=" * 80)

print("\n📊 Phase 7 Summary:")
print("  ✅ Notebook 1: FastAPI Production API")
print("  ✅ Notebook 2: Docker & Cloud Deployment")
print("  ✅ Notebook 3: Monitoring & Analytics")

print("\n🎯 Achievements:")
print("  - Production REST API (8 endpoints)")
print("  - Docker containerization (multi-stage)")
print("  - Multi-service orchestration (5 services)")
print("  - Cloud deployment ready (AWS + GCP)")
print("  - Complete monitoring (Prometheus + Grafana)")
print("  - Structured logging (JSON)")
print("  - Alert rules (6 alerts)")
print("  - Performance analytics")

print("\n📍 Next Phase: Phase 8 - LLM Chat & Multi-turn")

print("\n" + "=" * 80)
print("🎊 PHASE 7, NOTEBOOK 3 COMPLETE!")
print("=" * 80)


🎯 QUALITY GATES VALIDATION
✅ Gate 1: Prometheus config created
✅ Gate 2: Alert rules created (6 alerts)
✅ Gate 3: Grafana dashboard created (6 panels)
✅ Gate 4: Logging config created (JSON format)
✅ Gate 5: Docker Compose monitoring created
✅ Gate 6: Analytics script created
✅ Gate 7: Monitoring docs created
✅ Gate 8: Monitoring directories structure complete

📊 Gates Passed: 8/8

🎉 QUALITY GATES PASSED!
✅ Phase 7, Notebook 3 complete!

📊 Summary:
  Prometheus config: ✓
  Alert rules: 6 ✓
  Grafana dashboard: 6 panels ✓
  Logging config: JSON ✓
  Docker Compose: monitoring ✓
  Analytics script: Python ✓
  Documentation: Complete ✓

📝 Monitoring Stack:

# Start monitoring:
cd docker
docker-compose -f docker-compose.yml -f docker-compose.monitoring.yml up -d

# Access:
Prometheus: http://localhost:9090
Grafana: http://localhost:3000 (admin/admin)

# Run analytics:
python monitoring/analytics.py

# View logs:
docker-compose logs -f api


🎊 PHASE 7 COMPLETE! ALL 3 NOTEBOOKS DONE!

📊 Phas

---

## 📋 Summary

**Phase 7, Notebook 3 Complete!** ✅

### Achievements:

**1. Prometheus Metrics**
- Configuration file
- Scrape jobs (API, Redis, Nginx, Node)
- 15s scrape interval
- Alert manager integration

**2. Alert Rules (6 Total)**
- HighErrorRate (>5%)
- HighLatency (P95 >1s)
- ServiceDown (>1min)
- HighCPUUsage (>80%)
- HighMemoryUsage (>85%)
- RedisDown (>1min)

**3. Grafana Dashboard**
- 6 panels
- Request rate
- Error rate
- Latency (P50, P95, P99)
- CPU/Memory usage
- Redis operations

**4. Structured Logging**
- JSON format
- Console + File handlers
- Rotating files (10MB, 5 backups)
- Separate error log

**5. Docker Compose Monitoring**
- Prometheus service
- Grafana service
- Persistent volumes
- Network integration

**6. Performance Analytics**
- Python script
- Prometheus API queries
- 24h reports
- Automated analysis

**7. Documentation**
- Complete monitoring guide
- Architecture overview
- Usage instructions
- Troubleshooting
- SLA targets

### Files Created:

```
monitoring/
├── prometheus/
│   └── prometheus.yml
├── grafana/
│   └── dashboard.json
├── alerts/
│   └── api_alerts.yml
├── analytics.py
└── README.md

config/
└── logging.yaml

docker/
└── docker-compose.monitoring.yml
```

### Monitoring Stack:

**Services:**
- Prometheus (Port 9090)
- Grafana (Port 3000)

**Metrics:**
- Request rate, latency, errors
- CPU, memory, disk
- Redis operations

**Alerts:**
- Critical: Service down, high errors
- Warning: High latency, resource usage

### SLA Targets:

**Availability:** 99.9%  
**Latency P95:** <500ms  
**Error Rate:** <1%

---

## 🎊 PHASE 7 COMPLETE!

**All 3 Notebooks Done:**
1. ✅ FastAPI Production API
2. ✅ Docker & Cloud Deployment
3. ✅ Monitoring & Analytics

**Total Achievements:**
- Production REST API ✅
- Docker containerization ✅
- Cloud deployment ready ✅
- Complete monitoring ✅
- Observability stack ✅

**Next:** Phase 8 - LLM Chat & Multi-turn

---